In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping


In [ ]:
data_path = "data/"

X_train = np.load(f"{data_path}/single/X_train.npy")
X_test = np.load(f"{data_path}/single/X_test.npy")
y_train = np.load(f"{data_path}/single/y_train.npy")
y_test = np.load(f"{data_path}/single/y_test.npy")

In [ ]:
# Declares fews variables

img_shape = (32, 32, 1)
n_classes = 10
n_filters = 16
n_epochs = 20 # How many time the CNet will be feed
kernel_size = (4, 4) # Shape of the filter, le filtre correspond une petite matrice utilisé pour appliqué un produit matriciel et simplifier les données
pool_size = (2, 2) # Shape of the feature after the pooling
batch_size = 64 # NB. samples in a batch. Batch mean a subset of X since we not gonna feed the CNet once

In [ ]:
inputs = Input(shape=img_shape)

cov = Conv2D(n_filters,kernel_size=(kernel_size[0],kernel_size[1]),padding='same', use_bias=False)(inputs)
cov = BatchNormalization()(cov)
cov = Activation('relu')(cov)
cov = Conv2D(n_filters,kernel_size=(kernel_size[0],kernel_size[1]),padding='same', use_bias=False)(cov)
cov = BatchNormalization()(cov)
cov = Activation('relu')(cov)
cov = MaxPooling2D(pool_size=pool_size)(cov)
cov = Dropout(0.3)(cov)

cov = Conv2D((n_filters * 2),kernel_size=(kernel_size[0],kernel_size[1]), use_bias=False)(cov)
cov = BatchNormalization()(cov)
cov = Activation('relu')(cov)
cov = Conv2D((n_filters * 2),kernel_size=(kernel_size[0],kernel_size[1]),padding='same', use_bias=False)(cov)
cov = BatchNormalization()(cov)
cov = Activation('relu')(cov)
cov = MaxPooling2D(pool_size=pool_size)(cov)
cov = Dropout(0.3)(cov)


cov = Conv2D((n_filters * 4),kernel_size=(kernel_size[0],kernel_size[1]), use_bias=False)(cov)
cov = BatchNormalization()(cov)
cov = Activation('relu')(cov)
cov = Conv2D((n_filters * 4),kernel_size=(kernel_size[0],kernel_size[1]),padding='same', use_bias=False)(cov)
cov = BatchNormalization()(cov)
cov = Activation('relu')(cov)
cov = MaxPooling2D(pool_size=pool_size)(cov)
cov = Dropout(0.3)(cov)

cov = Conv2D((n_filters * 8),kernel_size=(kernel_size[0],kernel_size[1]), use_bias=False, padding="same")(cov)
cov = BatchNormalization()(cov)
cov = Activation('relu')(cov)
cov = Conv2D((n_filters * 8),kernel_size=(kernel_size[0],kernel_size[1]),padding='same', use_bias=False)(cov)
cov = BatchNormalization()(cov)
cov = Activation('relu')(cov)
cov = MaxPooling2D(pool_size=pool_size, padding="same")(cov)
cov = Dropout(0.3)(cov)

cov_out = Flatten()(cov)


# Dense Layers
cov2 = Dense(2056, activation='relu')(cov_out)
cov2 = Dropout(0.3)(cov2)



# Prediction layers
c0 = Dense(n_classes, activation='softmax')(cov2)

# Defining the model
model = Model(inputs=inputs,outputs=c0)
print (model.summary())

In [ ]:
print('Training set', X_train.shape, y_train.shape)
print('Test set', X_test.shape, y_test.shape)

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')

# Fitting the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=n_epochs,verbose=1,
          validation_data=(X_test, y_test), callbacks=[early_stopping])

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
correct = np.argmax(y_pred, axis=1) == np.argmax(y_test, axis=1)
incorrect = np.argmax(y_pred, axis=1) != np.argmax(y_test, axis=1)
accuracy = sum(correct)/correct.shape[0]
print(f"Accuracy's model {round(accuracy*100, 2)} %")

In [ ]:
def plot_result(imgs, y_true_p, y_pred_p, veracity=True):
    fig, axes = plt.subplots(1, 10, figsize=(25, 25))
    axes = axes.flatten()
    X_plot = imgs[correct] if veracity else imgs[incorrect]
    y_true_p = y_true_p[correct] if veracity else y_true_p[incorrect]
    y_pred_p = y_pred_p[correct] if veracity else y_pred_p[incorrect]
    for i, ax in enumerate(axes):
        ax.imshow(X_plot[i,:,:,0], cmap="binary")

        title = f"True:{np.argmax(y_true_p[i])}, Pred:{np.argmax(y_pred_p[i])}"
        ax.set_title(title)

In [ ]:
# Plot somes good result
plot_result(X_test, y_test, y_pred)

In [ ]:
# Plot somes wrong result
plot_result(X_test, y_test, y_pred, veracity=False)

In [ ]:
from sklearn.metrics import confusion_matrix

# Set the figure size
plt.figure(figsize=(12, 8))

# Calculate the confusion matrix
cm = confusion_matrix(y_true=np.argmax(y_test, axis=1), y_pred=np.argmax(y_pred, axis=1))

# Normalize the confusion matrix
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100.0

# Visualize the confusion matrix
sns.heatmap(cm, annot=True, cmap='Reds', fmt='.1f', square=True);